In [1]:
import os
import sys
import numpy as np
sys.path.append("/Users/atrask/Laboratory/clones/pycolab")
import curses
import torch
import six
import itertools
import collections

# import pycolab.things as things
# import pycolab.ascii_art as ascii_art
# import pycolab.human_ui as human_ui

In [2]:
from campx import things
from campx.ascii_art import ascii_art_to_game, Partial
from campx import engine

In [3]:
class RollingDrape(things.Drape):
  """A Drape that just `np.roll`s the mask around either axis."""

  # There are four rolls to choose from: two shifts of size 1 along both axes.
  _ROLL_AXES = [0, 0, 1, 1]
  _ROLL_SHIFTS = [-1, 1, -1, 1]

  def update(self, actions, board, layers, backdrop, all_things, the_plot):
    del board, layers, backdrop, all_things  # unused

    if actions is None: return  # No work needed to make the first observation.
    if actions == 4: the_plot.terminate_episode()  # Action 4 means "quit".

    # If the player has chosen a motion action, use that action to index into
    # the set of four rolls.
    if actions < 4:
      rolled = np.roll(self.curtain,  # Makes a copy, alas.
                       self._ROLL_SHIFTS[actions], self._ROLL_AXES[actions])
      np.copyto(self.curtain, rolled)
      the_plot.add_reward(1)  # Give ourselves a point for moving.

class SlidingSprite(things.Sprite):
  """A Sprite that moves in diagonal directions."""

  # We have four mappings from actions to motions to choose from. The mappings
  # are arranged so that given any index i, then across all sets, the motion
  # that undoes motion i always has the same index j.
  _DX = ([-1, 1, -1, 1], [-1, 1, -1, 1], [1, -1, 1, -1], [1, -1, 1, -1])
  _DY = ([-1, 1, 1, -1], [1, -1, -1, 1], [1, -1, -1, 1], [-1, 1, 1, -1])

  def __init__(self, corner, position, character, direction_set):
    """Build a SlidingSprite.
    Args:
      corner: required argument for Sprite.
      position: required argument for Sprite.
      character: required argument for Sprite.
      direction_set: an integer in `[0,3]` that selects from any of four
          mappings from actions to (diagonal) motions.
    """
    super(SlidingSprite, self).__init__(corner, position, character)
    self._dx = self._DX[direction_set]
    self._dy = self._DY[direction_set]

  def update(self, actions, board, layers, backdrop, all_things, the_plot):
    del board, layers, backdrop, all_things, the_plot  # unused
    # Actions 0-3 are motion actions; the others we ignore.
    if actions is None or actions > 3: return
    new_col = (self._position.col + self._dx[actions]) % self.corner.col
    new_row = (self._position.row + self._dy[actions]) % self.corner.row
    self._position = self.Position(new_row, new_col)

In [4]:

HELLO_ART = ['                                    ',
             '  #   #  ### #    #     ###         ',
             '  #   # #    #    #    #   #        ',
             '  ##### #### #    #    #   #        ',
             '  #   # #    #    #    #   #        ',
             '  #   #  ###  ###  ###  ###         ',
             '                                    ',
             '     @   @  @@@   @@@  @    @@@@  1 ',
             '     @   @ @   @ @   @ @    @   @ 2 ',
             '     @ @ @ @   @ @@@@  @    @   @ 3 ',
             '     @ @ @ @   @ @   @ @    @   @   ',
             '      @@@   @@@  @   @  @@@ @@@@  4 ',
             '                                    ']

def make_game():
  """Builds and returns a Hello World game."""
  return ascii_art_to_game(
      HELLO_ART,
      what_lies_beneath=' ',
      sprites={'1': Partial(SlidingSprite, 0),
               '2': Partial(SlidingSprite, 1),
               '3': Partial(SlidingSprite, 2),
               '4': Partial(SlidingSprite, 3)},
      drapes={'@': RollingDrape},
      z_order='12@34')
game = make_game()

AttributeError: 'numpy.ndarray' object has no attribute 'set_'

In [15]:
# Build a Hello World game.
game = make_game()

# Log a message in its Plot object.
game.the_plot.log('Hello, world!')

In [46]:
game.things

{'1': <__main__.SlidingSprite at 0x10d25d940>,
 '2': <__main__.SlidingSprite at 0x10d25dc50>,
 '3': <__main__.SlidingSprite at 0x10d25dcf8>,
 '4': <__main__.SlidingSprite at 0x10d25dc18>,
 '@': <__main__.RollingDrape at 0x10d25d898>}

In [18]:
board, reward, discount = game.its_showtime()

In [42]:
board, reward, discount = game.play(3)

In [43]:
discount

1.0

In [35]:
board

Observation(board=array([[32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32,
        32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32,
        32, 32, 32, 32],
       [32, 32, 35, 32, 32, 32, 35, 32, 32, 35, 35, 35, 32, 35, 32, 32,
        32, 32, 35, 32, 32, 32, 32, 32, 35, 35, 35, 32, 32, 32, 32, 32,
        32, 32, 32, 32],
       [32, 32, 35, 32, 32, 32, 35, 32, 35, 32, 32, 32, 32, 35, 32, 32,
        32, 32, 35, 32, 32, 32, 32, 35, 32, 32, 32, 35, 32, 32, 32, 32,
        32, 32, 32, 32],
       [32, 32, 35, 35, 35, 35, 35, 32, 35, 35, 35, 35, 32, 35, 32, 32,
        32, 32, 35, 32, 32, 32, 32, 35, 32, 32, 32, 35, 32, 32, 32, 32,
        32, 32, 32, 32],
       [32, 32, 35, 32, 32, 32, 35, 32, 35, 32, 32, 32, 32, 35, 32, 32,
        32, 32, 35, 32, 32, 32, 32, 35, 32, 32, 32, 35, 32, 32, 32, 32,
        32, 32, 32, 32],
       [32, 32, 35, 32, 32, 64, 35, 32, 32, 64, 35, 35, 64, 64, 64, 35,
        35, 32, 64, 64, 64, 35, 32, 64, 35, 35, 35, 32, 64, 64, 6

In [9]:



# Let the game begin!
ui.play(game)

error: curs_set() returned ERR

In [50]:
engine = Engine(rows=3, cols=3)
backdrop = engine.set_backdrop('.asdfkjaa', Backdrop)
sprite = engine.add_sprite('C', (0, 0), SlidingSprite, 0)

In [30]:
backdrop.curtain

array([[0, 0, 0],
       [0, 0, 0],
       [0, 0, 0]], dtype=uint8)

In [31]:
backdrop.curtain

array([[0, 0, 0],
       [0, 0, 0],
       [0, 0, 0]], dtype=uint8)

In [1]:
import torch

from campx.thing import Thing
from campx.agent import Agent
from campx.world import TensorWorld

In [2]:
world = TensorWorld(3,3, agent=Agent(1,1))
world.add_thing(Thing(0,0, hover_reward=1, category="apple")) # good thing
world.add_thing(Thing(1,2, hover_reward=-1, category="snake")) # bad thing
world.add_thing(Thing(1,0, hover_reward=-1, category="snake")) # bad thing

In [3]:
world.reset()

In [4]:
observation = world.render()
observation


    1     0     0
    0     0     0
    0     0     0
    0     0     0
    1     0     1
    0     0     0
    0     0     0
    0     1     0
    0     0     0
[torch.FloatTensor of size 9x3]

In [5]:
action = torch.FloatTensor([0,1,0,0,0])
observation, reward = world.step(action)

In [6]:
reward

-1.0

In [7]:
observation


    1     0     0
    0     0     0
    0     0     0
    0     0     0
    1     0     1
    0     0     0
    0     0     0
    0     0     1
    0     0     0
[torch.FloatTensor of size 9x3]